

*   https://it-start.online/articles/grafik-obnovljaemyj-v-realnom-vremeni-na-python - движущийся обновляющийся график
*   https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html - DQN
* https://habr.com/ru/articles/719544/




In [ ]:
# %%capture
!pip install --upgrade pip
!pip install gymnasium[classic_control] Cython  tensorflow  gym keras  keras-rl2   pyvirtualdisplay  pyglet
!pip install matplotlib
!pip install torch

In [ ]:
import gym
from gym import spaces
import numpy as np
import math
import random
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import tensorflow as tf

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Flatten
# from tensorflow.keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, Concatenate
# from keras.optimizers import Adam


from keras import __version__
tf.keras.__version__ = __version__
from tensorflow.keras.optimizers.legacy import Adam
from rl.memory import SequentialMemory
from rl.random import OrnsteinUhlenbeckProcess
from rl.agents import DDPGAgent

Используя датчик 7 в 1 для измерения показателей субстрата (земли с добавками), который будет использоваться в гидропонных системах выращивания. Сможем получать информацию от каждого растения:
* температура
* влажность субстрата;
* pH - кислотность;
* EC/TDS - общая минерализация;
* NPK количество
  * N азота
  * P фосфора
  * K калия.

Показатели почвы, которые влияют на рост и развитие картофеля, включают следующие:

1. Температура:
   - Оптимальная температура почвы для картофеля составляет примерно 13-16 градусов Цельсия.
   - Рост картофеля замедляется при температуре ниже 10 градусов Цельсия и прекращается при температуре ниже 5 градусов Цельсия.

2. Влажность:
   - Картофель требует умеренной влажности почвы для хорошего роста и развития.
   - Оптимальный уровень влажности почвы для картофеля составляет примерно 60-70% от влагоемкости почвы.
   - Недостаток влаги может привести к медленному росту и низкому урожаю, а избыток влаги может способствовать развитию гнили и заболеваний.

3. pH кислотность:
   - Почва для картофеля должна иметь pH в диапазоне 5.0-6.5.
   - Низкое pH подавляет доступность необходимых питательных веществ для растения, а высокое pH может привести к блокировке определенных элементов питания.

4. EC/TDS - общая минерализация:
   - EC (Electrical Conductivity) или TDS (Total Dissolved Solids) используется для измерения общей минерализации почвы.
   - Общая минерализация должна находиться в допустимых пределах, которые могут варьироваться в зависимости от вида почвы и требований картофеля.
   - Оптимальные значения общей минерализации (EC/TDS) в субстрате для выращивания картофеля могут варьироваться в зависимости от различных факторов, таких как тип почвы, климатические условия, сорт картофеля и метод выращивания. Обычно принимаются следующие рекомендации по значениям EC/TDS:

    - *Для начала выращивания картофеля*: рекомендуется значение EC/TDS примерно **1,2-1,5 мСм/см** (декасименс на метр, мкСм/см), что соответствует умеренной минерализации.

    - *Во время роста растений*: рекомендуется значение EC/TDS примерно **1,5-2 мСм/см** для поддержания оптимальных условий роста и развития картофеля.

    - *Перед сбором урожая*: рекомендуется снизить значение EC/TDS до около **0,8-1,2 мСм/см** для достижения лучшего качества и вкуса картофеля.

    - Важно отметить, что эти значения служат только ориентиром, и оптимальные показатели могут незначительно отличаться в зависимости от местных условий и предпочтений роста. Рекомендуется проведение агрохимического анализа почвы и консультация с агрономом или специалистом по грунту для определения конкретных требований и практик, которые следует применять при выращивании картофеля в вашем регионе.

5. NPK (азот, фосфор, калий):
   - Картофель требует определенного соотношения азота (N), фосфора (P) и калия (K) для нормального роста и развития.
   - Рекомендуемые значения NPK зависят от состояния почвы и могут быть получены через агрохимический анализ.
   - NPK относится к макрэлементам, которые являются основными питательными веществами для роста и развития растений. Они представлены следующими элементами: азот (N), фосфор (P) и калий (K). Для выращивания картофеля рекомендуется следующие NPK показатели:

    * Азот *(N)*: Азот необходим для развития листвы, стеблей и общей зеленой массы растения. Рекомендуется N-показатель примерно **120-150 кг/га** для картофеля в начале сезона и **60-90 кг/га** во время активного роста.

    * Фосфор *(P)*: Фосфор необходим для развития корней, цветения и формирования клубней. Рекомендуется P-показатель примерно **60-90 кг/га** для картофеля в начале сезона и **120-150 кг/га** во время активного роста.

    * Калий *(K)*: Калий необходим для образования и развития клубней, а также повышения стойкости растений к стрессу и болезням. Рекомендуется K-показатель примерно **120-150 кг/га** для картофеля в начале сезона и **150-180 кг/га** во время активного роста.

   - Эти значения являются общими рекомендациями и могут варьироваться в зависимости от типа почвы, условий выращивания и сорта картофеля. Рекомендуется проведение агрохимического анализа почвы и консультация с агрономом или специалистом по грунту для определения конкретных потребностей в NPK и разработки индивидуального плана питания для выращивания картофеля.


In [ ]:
def NRandom(n):
    # Генерируем первые два числа
    numbs = [
        random.uniform(0,1) for _ in range(n-1)
    ]
    numbs.append(1 - sum(numbs))
    if all([0 < x < 1 for x in numbs]):
      return np.array(numbs)
    else:
      return NRandom(n)

# Коэффициент теплопроводности (k): Коэффициент теплопроводности характеризует способность почвы проводить тепло. Для чернозема обычно принимают значение около 0.25 Вт/(м·К) при нормальных условиях (сухой почве и определенной температуре).

# Коэффициент проводимости влаги (K): Коэффициент проводимости влаги определяет, насколько быстро вода проникает в почву. Значение K также будет зависеть от многих факторов, но для чернозема оно может варьироваться от 0.05 до 0.5 см/час.

In [ ]:
class SoilSystemEnv(gym.Env):
  def __init__(
      self,
      optimal_space:dict,#{"Space_Var":[opt_start=-np.inf,opt_end=np.inf]},
      V:float=1, # объем горшка,
      alpha:float=0.2, # для расчета температуры почвы: T_soil` = T_air - alpha(T_air - T_soil) alpha in [0.1 0.3]
      beta:float=0.3, # для расчета влажности почвы: зрш` = phi_soil` + beta(phi_air - phi_soil)*100 alpha in [0.1 0.5]
      random_state:int=42,
      eps:float=1e-5,
      k:float=1e-3
      ):
        # Actions we can take, down, stay, up
        #  В качестве действия агент(ы) могут подкручивать три винтиля (состава растворов №1 и №2, количество добавляемой воды до 1 литра) +
        self.action_space = spaces.Box(
          low=np.array([  -1,  -1,  -1, -1,  -1]),
          high=np.array([ 1,  1,  1,  1,  1]), # Valve_NPK; Valve_pH|_EC_TDS; Valve_Water; T_air; phi_air
          shape=(5,),dtype=float) 
        self.observation_space = spaces.Box(
          low=
          np.array([0,  0,  0,    40*1e-5, 40*1e-5, 40*1e-5]), # T_soil phi_soil pH_soil EC_TDS N P K
          high=
          np.array([35,  1,  1,  200*1e-5,  200*1e-5,  200*1e-5]),
          shape=(6,),dtype=float)

        self.optimal_space = {
                # Soil
                'T_soil': np.array([optimal_space['T_soil'][0],optimal_space['T_soil'][1]]),
                'phi_soil': np.array([optimal_space['phi_soil'][0],optimal_space['phi_soil'][1]]),
                'pH_soil': np.array([optimal_space['pH_soil'][0],optimal_space['pH_soil'][1]]),
                # 'EC_TDS': np.array([optimal_space['EC_TDS'][0],optimal_space['EC_TDS'][1]]),
                'N': np.array([optimal_space['N'][0],optimal_space['N'][1]]),
                'P': np.array([optimal_space['P'][0],optimal_space['P'][1]]),
                'K': np.array([optimal_space['K'][0],optimal_space['K'][1]]),
        }
        self.state = self.observation_space.sample() 
        self.V = V
        self.k=k

        # Valve 1 params (random)
        self.valve_1_N = np.array([np.mean(self.optimal_space[param]) - np.min(np.concatenate([self.optimal_space[par] for  par in ['N','P','K']])) for param in ['N','P','K']]) / \
          (np.max(np.concatenate([self.optimal_space[par] for  par in ['N','P','K']])) - np.min(np.concatenate([self.optimal_space[par] for  par in ['N','P','K']]))) # N, P, K

        # Soil propereties
        self.alpha = alpha
        self.beta = beta
        self.eps = eps
        print(
            f'valve_1_N: {self.valve_1_N}'+
            # f'\nvalve_2_N: {self.valve_2_N}'+
            f'\nalpha: {self.alpha}'+
            f'\nbeta: {self.beta}'+
            
            f'\nSTATE_START:'+
            f'\n T_soil: {self.state[0]}'+#{self.state[["T_soil"]]}'+
            f'\n phi_soil: {self.state[1]}'+#{self.state["phi_soil"]}'+
            f'\n pH_soil: {self.state[2]}'+#{self.state["pH_soil"]}'+
            # f'\n EC_TDS: {self.state[3]}'+#{self.state["EC_TDS"]}'+
            f'\n [N P K]: [{self.state[3]} {self.state[4]} {self.state[5]}]'#{self.state["N"]} {self.state["P"]} {self.state["K"]}]'+
            
            f'\nSTATE_OPTIMAL:'+
            f'\n T_soil: {self.optimal_space["T_soil"]}'+
            f'\n phi_soil: {self.optimal_space["phi_soil"]}'+
            f'\n pH_soil: {self.optimal_space["pH_soil"]}'+
            # f'\n EC_TDS: {self.optimal_space["EC_TDS"]}'+
            f'\n [N P K]: [{self.optimal_space["N"]} {self.optimal_space["P"]} {self.optimal_space["K"]}]'
              )

  def step(self,action):
        # 1. Apply action
        #№  Get new params
        T_air = 18 + action[-2] * 12
        water = action[2] * self.V
        
        phi_air = action[-1]
        
        N_act, P_act, K_act = action[0] * self.valve_1_N * water
        pH_soil = action[1]#, EC_TDS = action[1] * self.valve_2_N*water
        # pH_soil = action[1]  

        # 2. Apply State
        ##  T_soil
        self.state[0] =  T_air - self.alpha * (T_air - self.state[0])
        ##  phi_soil
        self.state[1] =  phi_air + self.beta * (phi_air - self.state[1])
        self.state[1] = self.state[1] + water/self.V if self.state[1] + water/self.V < 1 else 1
        ## NPK
        self.state[3] += N_act + random.uniform(-.10,0)
        self.state[4] += P_act + random.uniform(-.10,0)
        self.state[5] += K_act + random.uniform(-.10,0)
        ## pH_soil, EC_TDS
        self.state[2] += pH_soil + random.uniform(-.10,0)
        # self.state[3] += EC_TDS + random.uniform(-1.0,0)

        # 3. Get reward
        # Вычисление вознаграждения
        reward = self._get_reward(self.state)

        # Определение, является ли эпизод завершенным
        if abs(reward - 1) <=self.eps:
          
          done=True
        else:
          done = False
        # print(f'self.reward={reward}')
        # Set placeholder for info
        info = {}

        # Return step information
        return self.state, reward, done, info
  def render(self):
    # Визуализация текущего состояния среды
    pass
  def _get_reward(self, state):     
      keys = ['T_soil','phi_soil','pH_soil','N','P','K']
      # reward_list = np.array([1 if self.optimal_space[keys[i]][0] <= state[i] <= self.optimal_space[keys[i]][1] else np.clip(np.exp(np.mean(self.optimal_space[keys[i]]) - state[i]),0,1) for i in range(len(keys))])  
      # reward_list = np.array([1 if self.optimal_space[keys[i]][0] <= state[i] <= self.optimal_space[keys[i]][1] else 0 for i in range(len(keys))])    
      clipped_reward = np.clip(np.mean(reward_list), 0, 1)
      return clipped_reward

  def reset(self):
    self.state =self.observation_space.sample()
    return self.state


In [ ]:
# # 1. Дискретные пространства состояний: Дискретные пространства состояний представляются целыми числами или наборами дискретных значений. Например, игры с сеткой, где состояния могут иметь конечное количество дискретных позиций, могут использовать дискретные пространства состояний.

# #    Примеры дискретных пространств состояний в Gym:
# #    - `spaces.Discrete(n)`: пространство из `n` дискретных состояний.
# #    - `spaces.MultiDiscrete(nvec)`: многомерное пространство состояний с `nvec` дискретными значениями по каждому измерению.

# # 2. Непрерывные пространства состояний: Непрерывные пространства состояний представляются вещественными числами или интервалами значений. Например, в задачах управления роботами или автономными автомобилями, где состояния могут быть описаны непрерывными переменными, можно использовать непрерывные пространства состояний.

# #    Примеры непрерывных пространств состояний в Gym:
# #    - `spaces.Box(low, high, shape)`: пространство состояний, представленное в виде прямоугольника с нижними (`low`) и верхними (`high`) границами значений переменных в каждом измерении.
# #    - `spaces.MultiBox([spaces.Box(), spaces.Box(), ...])`: многомерное пространство состояний, состоящее из нескольких непрерывных пространств состояний.
# # Например, если начальная температура почвы T0 и окружающая среда имеет постоянную температуру T_env, то решение может иметь вид:

# # T(t) = T_env + (T0 - T_env) * exp(-kt)

# # где t - время.

# # В данном случае, значение коэффициента охлаждения k будет определяться конкретными условиями (материал горшка, его размеры и т.д.), а начальная температура почвы и температура окружающей среды будут параметрами, которые следует учесть в оценке остывания почвы.

# class SoilSystemEnv(gym.Env):
#   def __init__(
#       self,
#       optimal_space:dict,#{"Space_Var":[opt_start=-np.inf,opt_end=np.inf]},
#       V:float=1, # объем горшка,
#       alpha:float=0.2, # для расчета температуры почвы: T_soil` = T_air - alpha(T_air - T_soil) alpha in [0.1 0.3]
#       beta:float=0.3, # для расчета влажности почвы: зрш` = phi_soil` + beta(phi_air - phi_soil)*100 alpha in [0.1 0.5]
#       random_state:int=42
#       ):
#         # Actions we can take, down, stay, up
#         #  В качестве действия агент(ы) могут подкручивать три винтиля (состава растворов №1 и №2, количество добавляемой воды до 1 литра) +
#         self.action_space = spaces.Box(low=np.array([0 for _ in range(5)]), high=np.array([1 for _ in range(5)]),shape=(5,),dtype=float)
#             # Valve_NPK; Valve_pH_EC_TDS; Valve_Water; T_air; phi_air
#         self.observation_space = spaces.Box(low=np.array([-100,0,0,0,0,0,0]),high=np.array([100,1,14,np.inf,np.inf,np.inf,np.inf]),shape=(7,),dtype=float)
#         #         'T_soil': random.uniform(-100, 100),
#         #         'phi_soil': random.uniform(0, 1),
#         #         'pH_soil': random.uniform(0, 14),
#         #         'EC_TDS': random.uniform(0.2, 5),
#         #         'N': random.uniform(0, 250),
#         #         'P': random.uniform(0, 250),
#         #         'K': random.uniform(0, 250)

#         self.optimal_space = {
#                 # Soil
#                 'T_soil': np.array([optimal_space['T_soil'][0],optimal_space['T_soil'][1]]),
#                 'phi_soil': np.array([optimal_space['phi_soil'][0],optimal_space['phi_soil'][1]]),
#                 'pH_soil': np.array([optimal_space['pH_soil'][0],optimal_space['pH_soil'][1]]),
#                 'EC_TDS': np.array([optimal_space['EC_TDS'][0],optimal_space['EC_TDS'][1]]),
#                 'N': np.array([optimal_space['N'][0],optimal_space['N'][1]]),
#                 'P': np.array([optimal_space['P'][0],optimal_space['P'][1]]),
#                 'K': np.array([optimal_space['K'][0],optimal_space['K'][1]]),
#         }
#         # Set start temp
#         self.state = self.observation_space.sample() #{
#         #         'T_soil': random.uniform(-100, 100),
#         #         'phi_soil': random.uniform(0, 1),
#         #         'pH_soil': random.uniform(0, 14),
#         #         'EC_TDS': random.uniform(0.2, 5),
#         #         'N': random.uniform(0, 250),
#         #         'P': random.uniform(0, 250),
#         #         'K': random.uniform(0, 250)

#         # }
#         # V - объем
#         self.V = V

#         # Valve 1 params (random)
#         self.valve_1_N = NRandom(3) # N, P, K
#         # Valve 2 params (random)
#         self.valve_2_N = NRandom(2) # pH_soil, EC_TDS

#         # Soil propereties
#         self.alpha = alpha
#         self.beta = beta
#         print(
#             f'valve_1_N: {self.valve_1_N}'+
#             f'\nvalve_2_N: {self.valve_2_N}'+
#             f'\nalpha: {self.alpha}'+
#             f'\nbeta: {self.beta}'+
#             f'\nSTATE_START:'+
#             f'\n T_soil: {self.state[0]}'+#{self.state[["T_soil"]]}'+
#             f'\n phi_soil: {self.state[1]}'+#{self.state["phi_soil"]}'+
#             f'\n pH_soil: {self.state[2]}'+#{self.state["pH_soil"]}'+
#             f'\n EC_TDS: {self.state[3]}'+#{self.state["EC_TDS"]}'+
#             f'\n [N P K]: [{self.state[4]} {self.state[5]} {self.state[6]}]'#{self.state["N"]} {self.state["P"]} {self.state["K"]}]'+
#             f'\nSTATE_OPTIMAL:'+
#             f'\n T_soil: {self.optimal_space["T_soil"]}'+
#             f'\n phi_soil: {self.optimal_space["phi_soil"]}'+
#             f'\n pH_soil: {self.optimal_space["pH_soil"]}'+
#             f'\n EC_TDS: {self.optimal_space["EC_TDS"]}'+
#             f'\n [N P K]: [{self.optimal_space["N"]} {self.optimal_space["P"]} {self.optimal_space["K"]}]'
#               )

#   def step(self,action):
#         # 1. Apply action
#         #№  Get new params
#         T_air = 18 + action[-2] * 12
#         phi_air = action[-1]
#         N_act,P_act,K_act = action[0] * self.valve_1_N
#         pH_soil, EC_TDS = action[1] * self.valve_2_N
#         water = action[2]

#         # 2. Apply State
#         ##  T_soil
#         self.state[0] =  T_air - self.alpha * (T_air - self.state[0])
#         ##  phi_soil
#         self.state[1] =  phi_air + self.beta * (phi_air - self.state[1])
#         self.state[1] = self.state[1] + water/self.V if self.state[1] + water/self.V < 1 else 1
#         ## NPK
#         self.state[4] += N_act + random.uniform(-1.0,0)
#         self.state[5] += P_act + random.uniform(-1.0,0)
#         self.state[6] += K_act + random.uniform(-1.0,0)
#         ## pH_soil, EC_TDS
#         self.state[2] += pH_soil + random.uniform(-1.0,0)
#         self.state[3] += EC_TDS + random.uniform(-1.0,0)

#         # 3. Get reward
#         # Вычисление вознаграждения
#         reward = self._get_reward(self.state)

#         # Определение, является ли эпизод завершенным
#         done = False
#         # Set placeholder for info
#         info = {}

#         # Return step information
#         return self.state, reward, done, info
#   def render(self):
#     # Визуализация текущего состояния среды
#     pass

#   def _get_reward(self, state:np.array):
#         optimal_means = np.array([np.mean(self.optimal_space[key]) for key in self.optimal_space.keys()])
#         # current_means = np.array([state[key] for key in state.keys()])
#         return 1 - np.mean((optimal_means - state)**2)
#       # if all([(self.optimal_space[key][0] <=state[key] <= self.optimal_space[key][1]).tolist()[0] for key in state.keys()]):
#       #   return 1
#       # else:
#       #   optimal_means = np.array([np.mean(self.optimal_space[key]) for key in state.keys()])
#       #   current_means = np.array([np.mean(state[key]) for key in state.keys()])
#       #   return 1 - np.mean((optimal_means - current_means)**2)
#   def reset(self):
#     self.state =self.observation_space.sample()
#     return self.state


In [ ]:
optimal_space_1 = {
                # Soil
                'T_soil': np.array([13,16]),
                'phi_soil': np.array([0.60,0.70]),
                'pH_soil': np.array([5,6.5]),
                'N': np.array([120*1e-5,150*1e-5]),
                'P': np.array([60*1e-5,90*1e-5]),
                'K': np.array([120*1e-5,150*1e-5]),
                'EC_TDS': np.array([1.2,1.5]),
        }

In [ ]:
env = SoilSystemEnv(optimal_space=optimal_space_1)

## DQN

### PyTorch

In [ ]:
class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations[0], 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions[0])

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [ ]:
states = env.observation_space.shape#env.observation_space.shape
actions = env.action_space.shape

In [ ]:
model_dqn = DQN(states,actions)

In [ ]:
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# BATCH_SIZE is the number of transitions sampled from the replay buffer
# GAMMA is the discount factor as mentioned in the previous section
# EPS_START is the starting value of epsilon
# EPS_END is the final value of epsilon
# EPS_DECAY controls the rate of exponential decay of epsilon, higher means a slower decay
# TAU is the update rate of the target network
# LR is the learning rate of the ``AdamW`` optimizer
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

# Get number of actions from gym action space
n_actions = env.action_space.shape
# Get the number of state observations
state, info = env.reset()
n_observations = env.observation_space.shape

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0

In [ ]:
import numpy as np

# Размерность состояния
state_dim = 3
# Размерность действий
action_dim = 2

# Генерация случайного вектора состояния
state = np.random.rand(state_dim)
print("Состояние:", state)

# Генерация случайного вектора действий
actions = np.random.rand(action_dim)
print("Действия:", actions)

# Вычисление оценок действий для каждой компоненты состояния
q_values = np.zeros((state_dim, action_dim))
for i in range(state_dim):
    for j in range(action_dim):
        # Простой пример: оценка Q-value - сумма компонент состояния и действия
        q_values[i, j] = np.sum(state[i] * actions[j])

print("Оценки Q-value:")
print(q_values)

# Выбор наилучших действий для каждой компоненты состояния
best_actions = np.argmax(q_values, axis=0)
print("Наилучшие действия для каждой компоненты состояния:", best_actions)

In [ ]:
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        # Случайно выбираем действие с вероятностью epsilon
        action = np.random.rand(num_actions)
    else:
        # Иначе выбираем действие с наибольшей оценкой Q
        # Это предполагает, что у вас есть оценки Q для каждой компоненты действия
        q_values = calculate_q_values(state)  # Замените на вашу реализацию оценок Q
        action = np.argmax(q_values)

    return action


episode_durations = []


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

#### Training loop

In [ ]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0]
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [ ]:
if torch.cuda.is_available():
    num_episodes = 600
else:
    num_episodes = 50

for i_episode in range(num_episodes):
    # Initialize the environment and get it's state
    state, info = env.reset()
    print(state,info)
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    print(state,info)
    for t in count():
        action = select_action(state)
        print(action)
        observation, reward, terminated, truncated, _ = env.step(action)
        reward = torch.tensor([reward], device=device)
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the policy network)
        optimize_model()

        # Soft update of the target network's weights
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)

        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break

print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()

### [From git](https://github.com/nicknochnack/OpenAI-Reinforcement-Learning-with-Custom-Environment/blob/main/OpenAI%20Custom%20Environment%20Reinforcement%20Learning.ipynb) (TensorFlow)

In [ ]:
%%capture
!pip install --upgrade tensorflow
!pip install keras-rl2

Build Agent with Keras-RL

In [ ]:
def build_agent(
    env,
    gamma:float=.89,
    batch_size:int=64,
    target_model_update:float=1e-3,
    MemoryLimit:int=100000,
    theta:float=0.15
    ):
    actor = Sequential()
    actor.add(Flatten(input_shape=(1,) + env.observation_space.shape))
    actor.add(Dense(24, activation='relu'))
    actor.add(Dense(16, activation='relu'))
    actor.add(Dense(env.action_space.shape[0], activation='sigmoid'))

    # Определение архитектуры критика
    action_input = Input(shape=(env.action_space.shape[0],))
    observation_input = Input(shape=(1,) + env.observation_space.shape)
    flattened_observation = Flatten()(observation_input)

    x = Concatenate()([action_input, flattened_observation])
    x = Dense(32, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    x = Dense(1, activation='linear')(x)
    critic = Model(inputs=[action_input, observation_input], outputs=x)

    # Определение параметров и создание агента
    memory = SequentialMemory(limit=MemoryLimit, window_length=1)
    random_process = OrnsteinUhlenbeckProcess(size=env.action_space.shape[0],theta=theta)
    agent = DDPGAgent(    actor=actor,
        critic=critic,    critic_action_input=action_input,
        memory=memory,    nb_actions=env.action_space.shape[0],
        random_process=random_process,    gamma=gamma,
        batch_size=batch_size,    target_model_update=target_model_update
    )
    return agent

In [ ]:
agent = build_agent(env)
# Компиляция агента
agent.compile(Adam(learning_rate=1e-6, clipnorm=1.0), metrics=['mae'])
# Обучение агента (здесь вы должны использовать свои данные обучения)
agent.fit(env, nb_steps=100000, visualize=False, verbose=1)

In [ ]:
from itertools import product  
# GridSearch 
param_grid = { 
    'gamma': np.linspace(0.75, 0.99).tolist(), 
    'target_model_update': np.linspace(1e-2 - (1e-2)/4, 1e-3 + (1e-2)/4).tolist(), 
    'MemoryLimit': list(map(int, range(100000, 120000, 10000))), 
    'theta': np.linspace(0.05,0.15).tolist()
} 
 
# Создайте список всех комбинаций параметров 
param_combinations = list(product(*param_grid.values())) 
param_names = list(param_grid.keys()) 
 
# Инициализируйте переменные для хранения наилучших результатов 
best_reward = float('-inf') 
best_params = None 
best_model = None 
 
test_state = env.observation_space.sample() 
n_steps = 20 
 
# Итерируйтесь по всем комбинациям параметров 
for params in param_combinations: 
    try:
        agent_params = dict(zip(param_names, params)) 
        print(f'params: {agent_params}')
        agent = build_agent(env, **agent_params) 
        agent.compile(Adam(learning_rate=1e-6, clipnorm=1.0), metrics=['mae']) 
        agent.memory = SequentialMemory(limit=agent_params['MemoryLimit'], window_length=1) 
        
        # Обучение агента (здесь вы должны использовать свои данные обучения) 
        agent.fit(env, nb_steps=100000, visualize=False, verbose=1)

        # Test
        EPS = 1e-2
        state = test_state
        step=1
        
        action = agent.forward(state)
        new_observation, reward, done, info = env.step(action)
        
        while abs(reward - 1) > EPS or step < n_steps:
            action = agent.forward(state)
            new_observation, reward, done, info = env.step(action)
            state = new_observation
            step += 1
            
        print(f'reward = {reward}')
        
        
        # Проверьте, является ли текущая модель лучшей 
        if reward > best_reward: 
            best_reward = reward 
            best_params = agent_params 
            best_model = agent 
    except Exception as e:
        print(e)


In [ ]:
# Выведите наилучшие параметры и результаты 
print("Best Parameters:", best_params) 
print("Best reward:", best_reward)

In [ ]:
# # Пример выполнения действия с агентом# 
action = agent.forward(env.observation_space.sample())
new_observation, reward, done, info = env.step(action)
# Сохранение модели агента# 
agent.save_weights('ddpg_SOIL_weights.h5f')
# Загрузка модели агента
agent.load_weights('ddpg_SOIL_weights.h5f')
# Теперь агент готов для 

In [ ]:
start_state=env.observation_space.sample()
n_steps = 20
state = start_state
print(
        f'\nSTATE_OPTIMAL:'+
                f'\n T_soil: {env.optimal_space["T_soil"]}'+
                f'\n phi_soil: {env.optimal_space["phi_soil"]}'+
                f'\n pH_soil: {env.optimal_space["pH_soil"]}'+
                # f'\n EC_TDS: {env.optimal_space["EC_TDS"]}'+
                f'\n [N P K]: [{env.optimal_space["N"]} {env.optimal_space["P"]} {env.optimal_space["K"]}]'+
                f'\n state:'+
                        f'\n T_soil: {state[0]}'+#{self.state[["T_soil"]]}'+
                        f'\n phi_soil: {state[1]}'+#{self.state["phi_soil"]}'+
                        f'\n pH_soil: {state[2]}'+#{self.state["pH_soil"]}'+
                        # f'\n EC_TDS: {new_observation[3]}'+#{self.state["EC_TDS"]}'+
                        f'\n [N P K]: [{state[3]} {state[4]} {state[5]}]\n'+#{self.state["N"]} {self.state["P"]} {self.state["K"]}]' 
                '\n----------------------------------------------'
                ) 
EPS = 1e-2
if abs(env._get_reward(state) - 1) > EPS:
        for step in range(n_steps):
                print(f'reward = {env._get_reward(state)}')
                action = agent.forward(state)
                new_observation, reward, done, info = env.step(action)
                print(f'step {step}\n'+
                f' start_state:'+
                        f'\n T_soil: {state[0]}'+#{self.state[["T_soil"]]}'+
                        f'\n phi_soil: {state[1]}'+#{self.state["phi_soil"]}'+
                        f'\n pH_soil: {state[2]}'+#{self.state["pH_soil"]}'+
                        # f'\n EC_TDS: {state[3]}'+#{self.state["EC_TDS"]}'+
                        f'\n [N P K]: [{state[3]} {state[4]} {state[5]}]\n'#{self.state["N"]} {self.state["P"]} {self.state["K"]}]'+
                        
                f'\n------------------------------------\n action:\n'+
                f'\n new_observation:'+
                        f'\n T_soil: {new_observation[0]}'+#{self.state[["T_soil"]]}'+
                        f'\n phi_soil: {new_observation[1]}'+#{self.state["phi_soil"]}'+
                        f'\n pH_soil: {new_observation[2]}'+#{self.state["pH_soil"]}'+
                        # f'\n EC_TDS: {new_observation[3]}'+#{self.state["EC_TDS"]}'+
                        f'\n [N P K]: [{new_observation[3]} {new_observation[4]} {new_observation[5]}]\n'+#{self.state["N"]} {self.state["P"]} {self.state["K"]}]'+
                f'\n reward={reward}'+
                f'\n done={done}'+
                f'\n info={info}'+
                '\n----------------------------------------------'
                )
                state = new_observation
                if abs(env._get_reward(state) - 1) > EPS:
                        break        
else:
        print(f'reward = {env._get_reward(state)}')

## PPO

In [ ]:
%%capture
!pip install "stable-baselines3"
!pip install 'shimmy>=0.2.1'

In [ ]:
import torch as th
import torch.nn as nn

from stable_baselines3 import PPO
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

# Neural network for predicting action values
class CustomCNN(BaseFeaturesExtractor):

    def __init__(self, env, features_dim: int=128):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        # CxHxW images (channels first)
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with th.no_grad():
            n_flatten = self.cnn(
                th.as_tensor(observation_space.sample()[None]).float()
            ).shape[1]

        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations: th.Tensor) -> th.Tensor:
        return self.linear(self.cnn(observations))

In [ ]:
policy_kwargs = dict(
    features_extractor_class=CustomCNN,
)

# Initialize agent
model = PPO("CnnPolicy", env.observation_space, policy_kwargs=policy_kwargs, verbose=0)